### import the module needed

In [12]:
import sqlite3
import re
import csv


### helper functions

In [13]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except  sqlite3.Error as e:
        print(e)
        

def UnicodeDictReader(utf8_data, **kwargs):
    csv_reader = csv.DictReader(utf8_data, **kwargs)
    for row in csv_reader:
        yield {key: unicode(value, 'utf-8') for key, value in row.iteritems()}

## create a database for DC

In [14]:
created_db = sqlite3.connect("DC.db")

### database schema

In [15]:
schema_nodes= """
CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);"""

schema_node_tags="""
CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);"""
schema_ways="""
CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);"""
schema_way_tags="""

CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);"""
schema_way_nodes="""
CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);"""
schema_node_opening_hours="""
CREATE TABLE node_opening_hours (
    id INTEGER NOT NULL,
    hour TEXT,
    section INTEGER ,
    FOREIGN KEY (id) REFERENCES node_tags(id)
);"""
schema_way_opening_hours="""
CREATE TABLE way_opening_hours (
    id INTEGER NOT NULL,
    hour TEXT,
    section INTEGER ,
    FOREIGN KEY (id) REFERENCES way_tags(id)
);
"""

In [16]:
## list of schemas
schema=[schema_nodes,schema_node_tags,schema_node_opening_hours,schema_ways,schema_way_nodes,schema_way_tags,schema_way_opening_hours]


### drop existing tables

In [21]:
##drop all tables
drop_nodes='DROP TABLE nodes;'
drop_nodes_tags='DROP TABLE nodes_tags;'
drop_nodes_opening_hours='DROP TABLE node_opening_hours;'
drop_ways='DROP TABLE ways;'
drop_ways_nodes='DROP TABLE ways_nodes;'
drop_ways_tags='DROP TABLE ways_tags;'
drop_ways_opening_hours='DROP TABLE way_opening_hours;'
##as a list
drop=[drop_nodes,drop_nodes_tags,drop_nodes_opening_hours,
      drop_ways,drop_ways_nodes,drop_ways_tags,drop_ways_opening_hours]
for value in drop:
    create_table(created_db,value)

### create empty table

In [22]:
## call the function
for value in schema:
    create_table(created_db,value)

##commit the creation
created_db.commit()

In [23]:
##cursor
cursor=created_db.cursor()

In [24]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[(u'nodes',),
 (u'nodes_tags',),
 (u'node_opening_hours',),
 (u'ways',),
 (u'ways_nodes',),
 (u'ways_tags',),
 (u'way_opening_hours',)]

###  import csv file into table

#####  table nodes

In [25]:
with open('nodes.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['id'].decode('utf-8'), i['lat'].decode('utf-8'),i['lon'].decode('utf-8'),i['user'].decode('utf-8'),i['uid'].decode('utf-8'),i['version'].decode('utf-8'),i['changeset'].decode('utf-8'),i['timestamp'].decode('utf-8')) for i in dr]


cursor.executemany("INSERT INTO nodes(id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
created_db.commit()


#####  table nodes_tags

In [26]:
with open('nodes_tags.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db_node_tags = [(i['id'].decode('utf-8'), i['key'].decode('utf-8'),i['value'].decode('utf-8'),i['type'].decode('utf-8')) for i in dr]

cursor.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db_node_tags)
created_db.commit()

##### table node opening hours

In [27]:
with open('node_opening_hours.csv','rb') as fin:
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db_node_opening_hours= [(i['id'].decode('utf-8'), i['hour'].decode('utf-8'),i['section'].decode('utf-8')) for i in dr]

cursor.executemany("INSERT INTO node_opening_hours (id, hour, section) VALUES (?, ?, ?);", to_db_node_opening_hours)
created_db.commit()

#####  table ways

In [28]:
with open('ways.csv','rb')as fin:
    dr=csv.DictReader(fin)#id,user,uid,version,changeset,timestamp
    to_db_way=[(i['id'].decode('utf-8'),i['user'].decode('utf-8'),i['uid'].decode('utf-8'),i['version'].decode('utf-8'),i['changeset'].decode('utf-8'),i['timestamp'].decode('utf-8'))for i in dr]
    
    cursor.executemany("INSERT INTO ways (id,user,uid,version,changeset,timestamp) VALUES (?, ?, ?,?,?,?);", to_db_way)
created_db.commit()

##### table ways nodes

In [29]:
with open('ways_nodes.csv','rb')as fin:
    dr=csv.DictReader(fin)#id,node_id,position
    to_db_way_nodes=[(i['id'].decode('utf-8'),i['node_id'].decode('utf-8'),i['position'].decode('utf-8'))for i in dr]
    
    cursor.executemany("INSERT INTO ways_nodes (id,node_id,position) VALUES (?, ?, ?);", to_db_way_nodes)
created_db.commit()

###### table ways tags

In [30]:
with open('ways_tags.csv','rb')as fin:
    dr=csv.DictReader(fin)#id,key,value,type
    to_db_way_tags=[(i['id'].decode('utf-8'),i['key'].decode('utf-8'),i['value'].decode('utf-8'),i['type'].decode('utf-8'))for i in dr]
    
cursor.executemany("INSERT INTO ways_tags (id,key,value,type) VALUES (?, ?, ?,?);", to_db_way_tags)
created_db.commit()

##### table way opening hours

In [31]:
with open('way_opening_hours.csv','rb') as fin:
    dr = csv.DictReader(fin) # id,hour,section
    to_db_way_opening_hours= [(i['id'].decode('utf-8'), i['hour'].decode('utf-8'),i['section'].decode('utf-8')) for i in dr]

cursor.executemany("INSERT INTO way_opening_hours (id, hour, section) VALUES (?, ?, ?);", to_db_way_opening_hours)
created_db.commit()

### conduct queries

In [35]:
#cursor
#find the number of restraunts and group them by value for all
query_restraunt ='''select count(distinct id) as number_of_restraunt from 
                  (select id,key,value from nodes_tags union select id,key,value from ways_tags) where key="cuisine" ; 
                 '''
cursor.execute(query_restraunt)
result_restraunt=cursor.fetchall()
print result_restraunt

[(1441,)]


### categories for keys

In [48]:
#find all the categories
query_key='''
          select distinct key from ways_tags  ;
          '''
cursor.execute(query_key)
result_key=cursor.fetchall()
len(result_key)

400

### take a look at zipcode

In [55]:
query_postcode='''
                SELECT tags.key,tags.value, COUNT(*) as count 
                FROM (SELECT * FROM nodes_tags  UNION ALL 
                      SELECT * FROM ways_tags) tags
                WHERE tags.key = 'postal_code'
                GROUP BY tags.value
                ORDER BY count DESC;
               '''
cursor.execute(query_postcode)
result_postcode=cursor.fetchall()
result_postcode

[(u'postal_code', u'20744', 8),
 (u'postal_code', u'20740', 3),
 (u'postal_code', u'22207', 3),
 (u'postal_code', u'20745', 2),
 (u'postal_code', u'22201', 2),
 (u'postal_code', u'20005', 1),
 (u'postal_code', u'20009', 1),
 (u'postal_code', u'20735', 1),
 (u'postal_code', u'20770', 1),
 (u'postal_code', u'22202', 1),
 (u'postal_code', u'22204', 1),
 (u'postal_code', u'22206', 1),
 (u'postal_code', u'22209', 1)]

### count the number of cuisines in DC area

In [96]:
query_cuisine='''
                SELECT tags.value, COUNT(id) as count ,
                
                round((count(*)*100.0/(SELECT COUNT(*) FROM (
                    SELECT * FROM nodes_tags  UNION ALL 
                      SELECT * FROM ways_tags) t
                      WHERE t.key = 'cuisine'
                      ) ),2)
                      
                AS percetange
                      
                FROM (SELECT * FROM nodes_tags  UNION ALL 
                      SELECT * FROM ways_tags) tags
                WHERE tags.key = 'cuisine'
                GROUP BY tags.value
                ORDER BY count DESC
                limit 10;
               '''
cursor.execute(query_cuisine)
result_cuisine=cursor.fetchall()
result_cuisine

#ELECT a.name, a.gdppc, SUM(b.gdppc)
#FROM gdppc AS a, gdppc AS b WHERE b.gdppc <= a.gdppc 

[(u'pizza', 122, 8.47),
 (u'american', 118, 8.19),
 (u'sandwich', 111, 7.7),
 (u'burger', 110, 7.63),
 (u'coffee_shop', 96, 6.66),
 (u'mexican', 95, 6.59),
 (u'italian', 73, 5.07),
 (u'chinese', 70, 4.86),
 (u'thai', 57, 3.96),
 (u'indian', 35, 2.43)]

### number of restruants 

In [83]:
query_cuisine_number='''
                SELECT count(*)
                      
                FROM (SELECT * FROM nodes_tags  UNION ALL 
                      SELECT * FROM ways_tags) tags
                WHERE tags.key = 'cuisine';
               '''
cursor.execute(query_cuisine_number)
result_cuisine_number=cursor.fetchall()
result_cuisine_number

[(1441,)]

### Top 10 appearing amenities

In [99]:
query_amenity='''
               SELECT value, COUNT(distinct id) as num
               FROM nodes_tags
               WHERE key='amenity'
               GROUP BY value
               ORDER BY num DESC
               LIMIT 10;
               '''
cursor.execute(query_amenity)
result_amenity=cursor.fetchall()
result_amenity

[(u'restaurant', 1364),
 (u'place_of_worship', 811),
 (u'school', 772),
 (u'fast_food', 543),
 (u'bench', 527),
 (u'cafe', 469),
 (u'bank', 338),
 (u'bicycle_rental', 271),
 (u'drinking_water', 267),
 (u'bicycle_parking', 252)]

#### hours that are opened 

In [104]:
query_cuisine_opening='''
               SELECT tags.value, hours.hour, COUNT(distinct tags.id) as num 
               FROM (SELECT * FROM nodes_tags  UNION ALL 
                     SELECT * FROM ways_tags) tags join
                    (SELECT * FROM node_opening_hours  UNION ALL 
                      SELECT * FROM way_opening_hours) hours
                      on tags.id=hours.id
                      
               
               WHERE tags.key='cuisine'
               GROUP BY tags.value
               ORDER BY num DESC
               LIMIT 10;
               '''
cursor.execute(query_cuisine_opening)
result_cuisine_opening=cursor.fetchall()
result_cuisine_opening


[(u'sandwich',
  u'Sa 8:00 - 2:00, Su 8:00 - 21:00, Mo 8:30 - 21:00, Tu-Fr: 8:30 - 20:00',
  25),
 (u'american', u'Mo-Su 10:30-24:00', 21),
 (u'mexican', u'11:00-22:00', 17),
 (u'burger', u' Fr-Sa 10:00-03:30', 13),
 (u'pizza', u' Fr,Sa 00:00-01:00,11:00-24:00', 13),
 (u'chinese', u' su 12:00 - 20:00', 10),
 (u'italian', u'10:30 AM - 10:00 PM', 9),
 (u'thai', u' Su 12:00-15:00,16:30-21:00', 9),
 (u'coffee_shop', u' Su 07:00-22:00', 6),
 (u'mediterranean', u'su - th 11:00 - 21:00, fr-sa 11:00 - 22:00', 5)]

#####  suppose I would like to know on saturday where Can I find a Chinese restruant open and where they are

In [129]:
query_cuisine_chinese_opening='''
select tag.id,tag.value
from 
   (SELECT * FROM nodes_tags  UNION ALL 
    SELECT * FROM ways_tags) tag
                     
where tag.id in (select tags.id 
    FROM (SELECT * FROM nodes_tags  UNION ALL 
          SELECT * FROM ways_tags) tags join
         (SELECT * FROM node_opening_hours  UNION ALL 
          SELECT * FROM way_opening_hours) hours
          on tags.id=hours.id
    WHERE tags.key='cuisine' and tags.value='chinese'and hours.hour like"%su%"
    LIMIT 10)
    and tag.key like '%name%' 
;
'''

cursor.execute(query_cuisine_chinese_opening)
result_cuisine_chinese_opening=cursor.fetchall()
result_cuisine_chinese_opening



[(490254362, u'Mayflower'),
 (807592195, u'Great Wall Szechuan House'),
 (837122883, u'Nagomi Izakaya'),
 (3900521066, u'Eastern Carryout'),
 (296507739, u'Sammy Carry-Out'),
 (297371630, u"George's Carry Out"),
 (371641428, u'China House'),
 (417073480, u"Ho's Chinese Carry Out")]